In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select  # Select 클래스 import 추가
import pandas as pd
import time
import pyautogui
import pyperclip

options = Options()
# options.add_argument('start-maximized')         # 브라우저를 최대화된 상태로 시작
options.add_experimental_option('detach', True) # 스크립트가 종료된 후에도 브라우저가 닫히지 않도록 설정
# options.add_argument('headless')

driver = webdriver.Chrome(options=options)

driver.get("http://www.kipris.or.kr/khome/main.jsp")

time.sleep(1)
parent_window = driver.current_window_handle
# 모든 창의 핸들 가져오기
all_windows = driver.window_handles
for window in all_windows:
    if window != parent_window:
        driver.switch_to.window(window)         # 팝업 창 핸들로 전환
        driver.close()                          # 팝업 창 닫기 (팝업 창에서 필요한 동작 수행)
        driver.switch_to.window(parent_window)  # 부모 창으로 다시 전환
        break  


try:
    # 우측 상단의 '로그인' 버튼 클릭
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#gnb > li.cbp-hropen.depth1-1 > div > ul > li:nth-child(1) > a"))
    )
    login_button.click()
    time.sleep(1)

    ## Type Query Sentence
    elem=driver.find_element(By.ID, 'queryText')
    elem.send_keys('("AI"+"artificial intelligence"+"인공지능"+"인공 지능"+"머신러닝"+"machine learning")*AD=[20190624~20240624]')
    time.sleep(2)

    # 검색 버튼 클릭
    # <span class="input_btn"><a href="javascript:DoSearch()"><img src="/kpat/images/common/btn_search.gif" alt="검색"></a></span>
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".input_btn"))
    )
    search_button.click()
    time.sleep(1)
    
    select_element = driver.find_element(By.ID,'opt28')         # Select 요소를 찾아서 인스턴스화
    dropdown = Select(select_element)

    dropdown.select_by_value('90')                              # '페이지당 90개' 옵션 선택
    time.sleep(2)
    select_element = driver.find_element(By.CSS_SELECTOR,'#pageSel > a').click()        # 페이지당 특허 건 수 반영
    time.sleep(3)

    # 총 페이지 수 가져오기 
    articles_info = driver.find_element(By.CSS_SELECTOR, 'p.articles').text
    total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip())
    # print(f"Total pages: {total_pages}")

    patents_li=[];
    #for page_number in range(1, total_pages + 1):
    for page_number in range(1, 12):
        # 페이지 번호를 클릭하여 페이지 변경
        if page_number != 1 and page_number % 10 != 0:
            page_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f"//span[@class='board_pager03']//a[text()='{page_number}']"))
            )
            page_link.click()
            time.sleep(3)  # 페이지가 로드될 때까지 대기 
        elif page_number % 10 == 0:
            page_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="divMainArticle"]/form/section/div[2]/span/a[10]'))
            )
            page_link.click()
            time.sleep(3)  # 페이지가 로드될 때까지 대기 
        # 각 페이지의 특허 제목 추출
        titles = driver.find_elements(By.CSS_SELECTOR, 'h1.stitle')
        for index, title in enumerate(titles, start=1):
            #print(f"Title {index}: {title.text}")
            patents_li.append(title.text)


except Exception as e:
    print('An error occurred:', e)


finally:
    driver.quit()
df=pd.DataFrame(patents_li, columns=['Patent_Titles'])
df

An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6C08D3E32+31618]
	(No symbol) [0x00007FF6C084B099]
	(No symbol) [0x00007FF6C070888A]
	(No symbol) [0x00007FF6C0758524]
	(No symbol) [0x00007FF6C075862C]
	(No symbol) [0x00007FF6C079F787]
	(No symbol) [0x00007FF6C077D14F]
	(No symbol) [0x00007FF6C079CA80]
	(No symbol) [0x00007FF6C077CEB3]
	(No symbol) [0x00007FF6C074A46B]
	(No symbol) [0x00007FF6C074B001]
	GetHandleVerifier [0x00007FF6C0BD9FFD+3202381]
	GetHandleVerifier [0x00007FF6C0C26A1D+3516269]
	GetHandleVerifier [0x00007FF6C0C1C490+3473888]
	GetHandleVerifier [0x00007FF6C0985D36+760454]
	(No symbol) [0x00007FF6C0856B3F]
	(No symbol) [0x00007FF6C0851CD4]
	(No symbol) [0x00007FF6C0851E62]
	(No symbol) [0x00007FF6C084120F]
	BaseThreadInitThunk [0x00007FFEDFFE7344+20]
	RtlUserThreadStart [0x00007FFEE121CC91+33]



,Patent_Titles
0,"등록 [1] 갱신 영역 학습을 통한 인공지능 기반의 영상 분석 시스템 및 방법과, ..."
1,"등록 [2] 인공지능(Artificial Intelligence, AI) 알고리즘 ..."
2,취하 [3] 인공지능 돌봄 서비스 시스템(An AI Care Service System)
3,공개 [4] 인공지능 학습용 이미지 인식 데이터 품질관리를 활용한 자동화된 머신러닝...
4,"등록 [5] 머신러닝 기반 인공지능 모델 학습, 개발, 배포 및 운영 시스템과 이를..."
...,...
895,공개 [986] 인공지능을 활용한 고객통지 메시지 처리 시스템(Customer no...
896,공개 [987] 학습영상 데이터 기반의 인공지능 학습 지도 시스템(AI learni...
897,등록 [988] 복수의 인공지능 모델의 출력에 환경에 따른 가중치를 적용하는 비전 ...
898,등록 [989] 인공지능 기반 자기개선 게놈 시퀀싱을 통한 유전자 진단 시스템(SY...
